In [1]:
from my_tools import get_bill_data

In [2]:
data = get_bill_data()

------------------
------------------
Data is that with text from the 110th Congress (2007) to present
Alter masking in my_tools.get_bill_data to get a different data set.
------------------


In [3]:
data.shape

(52238, 22)

In [4]:
data.head()

,_id,bill_status,body,committee,congress_id,cosponsors,cosponsors_url,desc,intro_date,leg_id,...,num_of_cosponsors,sponsor,sponsor_district,sponsor_party,sponsor_state,bill_char_counts,intro_month,session,char_count_bucket,labels
0,5c182d561417de23a8260f58,Became Law,[110th Congress Public Law 180] [From the U.S....,House - Judiciary | Senate - Judiciary,110th,None,https://www.congress.gov/bill/110th-congress/h...,NICS Improvement Amendments Act of 2007,2007-06-11,H R 2640,...,17,"Rep. McCarthy, Carolyn",4,D,NY,432,6,2,less than 1000,1
1,5c182d561417de23a8261074,Became Law,[110th Congress Public Law 239] [From the U.S....,House - Judiciary | Senate - Judiciary,110th,None,https://www.congress.gov/bill/110th-congress/h...,"To amend title 4, United States Code, to encou...",2007-05-17,H R 2356,...,1,"Rep. Scott, David",13,D,GA,924,5,2,less than 1000,1
2,5c182d571417de23a82619aa,Became Law,[110th Congress Public Law 137] [From the U.S....,House - Appropriations,110th,None,None,Making further continuing appropriations for t...,2007-12-12,H J Res 69,...,0,"Rep. Obey, David R.",7,D,WI,797,12,2,less than 1000,1
3,5c182d581417de23a82621c3,Became Law,[110th Congress Public Law 80] [From the U.S. ...,"Senate - Agriculture, Nutrition, and Forestry",110th,None,https://www.congress.gov/bill/110th-congress/s...,"A bill to amend the U.S. Troop Readiness, Vete...",2007-06-27,S 1716,...,12,"Sen. Thune, John",None,R,SD,967,6,2,less than 1000,1
4,5c182d581417de23a8262651,Became Law,[110th Congress Public Law 201] [From the U.S....,Senate - Homeland Security and Governmental Af...,110th,None,https://www.congress.gov/bill/110th-congress/s...,A bill to preserve existing judgeships on the ...,2007-02-12,S 550,...,2,"Sen. Akaka, Daniel K.",None,D,HI,884,2,2,less than 1000,1


In [6]:
data.congress_id.value_counts()

110th    11220
111th    10770
112th    10612
114th    10223
113th     9082
115th      331
Name: congress_id, dtype: int64